## Goal

Speedup the writing to disk so that model prediction takes 90% of the time

In [8]:
import kipoi
import kipoi_veff.snv_predict as sp
from pathlib import Path
import os
from kipoi.writers import HDF5BatchWriter, TsvBatchWriter, MultipleBatchWriter
from kipoi_veff.utils.io import SyncBatchWriter

model_name = "DeepSEA/variantEffects"

In [11]:
cd notebooks/

/home/avsec/workspace/kipoi/kipoi-veff/notebooks


In [2]:
output_dir = Path("/tmp/kipoi")
output_dir.mkdir(exist_ok=True)

In [3]:
# Install line_profiler: https://github.com/rkern/line_profiler
# pip install line_profiler

In [4]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Then we need to know where the query VCF is located and where we want to store the results.

In [5]:
# The input vcf path
vcf_path = "example_data/clinvar_donor_acceptor_chr22.vcf"

Finally the dataloader arguments are set that are required to run the dataloader. Here we omit the `intervals_file` argument of the dataloader, because that has been tagged as bed file input in the `dataloader.yaml` file, which means that `score_variants` will automatically populate that argument with a temporary bed file that is generated from the VCF in order to query every variant contained in the input VCF. ("Variant-centered approach")

In [6]:
# The datalaoder keyword arguments
dataloader_arguments = {"fasta_file": "example_data/hg19_chr22.fa"}

### Writing to the vcf file

In [31]:
sp.score_variants(model = model_name,
                  dl_args = dataloader_arguments,
                  input_vcf = vcf_path,
                  output_vcf = str(output_dir / "output.vcf"))

Using downloaded and verified file: /home/avsec/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1
INFO [kipoi_veff.snv_predict] Using variant-centered sequence generation.


100%|██████████| 14/14 [00:02<00:00,  5.77it/s]


### Writing to a tsv file

**This is very slow (2 s / it)**

In [56]:
tsv_writer =  SyncBatchWriter(TsvBatchWriter(output_dir / "preds.tsv"))
sp.score_variants(model = model_name,
                  dl_args = dataloader_arguments,
                  input_vcf = vcf_path,
                  output_writers=tsv_writer,
                  output_vcf = None)

Using downloaded and verified file: /home/avsec/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1
INFO [kipoi_veff.snv_predict] Using variant-centered sequence generation.


100%|██████████| 14/14 [00:30<00:00,  2.19s/it]


### Writing to an hdf5 file

**This is slow as well**

In [39]:
h5_writer = SyncBatchWriter(HDF5BatchWriter(str(output_dir / 'preds.h5')))

In [40]:
sp.score_variants(model = model_name,
                  dl_args = dataloader_arguments,
                  input_vcf = vcf_path,
                  output_writers=h5_writer,
                  output_vcf = None)

Using downloaded and verified file: /home/avsec/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1
INFO [kipoi_veff.snv_predict] Using variant-centered sequence generation.


100%|██████████| 14/14 [00:27<00:00,  1.95s/it]


In [ ]:
# remove the file after running
!rm {h5_writer.batch_writer.file_path}

## Code profiling

Let's use line_profiler to see the bottlenecks in the code. Specify the function to benchmark with: `-f function`

In [58]:
tsv_writer =  SyncBatchWriter(TsvBatchWriter(output_dir / "preds.tsv"))
%lprun -f sp.predict_snvs sp.score_variants(model = model_name, dl_args=dataloader_arguments, input_vcf=vcf_path, output_writers=tsv_writer, output_vcf=None)

Using downloaded and verified file: /home/avsec/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1
INFO [kipoi_veff.snv_predict] Using variant-centered sequence generation.


100%|██████████| 14/14 [00:50<00:00,  3.64s/it]


Timer unit: 1e-06 s

Total time: 50.9212 s
File: /home/avsec/workspace/kipoi/kipoi-veff/kipoi_veff/snv_predict.py
Function: predict_snvs at line 468

Line #      Hits         Time  Per Hit   % Time  Line Contents
   468                                           def predict_snvs(model,
   469                                                            dataloader,
   470                                                            vcf_fpath,
   471                                                            batch_size,
   472                                                            num_workers=0,
   473                                                            dataloader_args=None,
   474                                                            vcf_to_region=None,
   475                                                            vcf_id_generator_fn=default_vcf_id_gen,
   476                                                            evaluation_function=analyse_model_preds,
   477       

You can see that line **656** takes 98% of the time. I figured out that the following function is making it slow: 

In [59]:
tsv_writer =  SyncBatchWriter(TsvBatchWriter(output_dir / "preds.tsv"))
%lprun -f tsv_writer.__call__ sp.score_variants(model = model_name, dl_args=dataloader_arguments, input_vcf=vcf_path, output_writers=tsv_writer, output_vcf=None)

Using downloaded and verified file: /home/avsec/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1
INFO [kipoi_veff.snv_predict] Using variant-centered sequence generation.


100%|██████████| 14/14 [00:51<00:00,  3.70s/it]


Timer unit: 1e-06 s

Total time: 50.7535 s
File: /home/avsec/workspace/kipoi/kipoi-veff/kipoi_veff/utils/io.py
Function: __call__ at line 362

Line #      Hits         Time  Per Hit   % Time  Line Contents
   362                                               def __call__(self, predictions, records, line_ids=None):
   363        14        307.0     21.9      0.0          validate_input(predictions, records, line_ids)
   364                                           
   365        14         16.0      1.1      0.0          if line_ids is None:
   366                                                       line_ids = {}
   367                                           
   368        14       4193.0    299.5      0.0          batch = numpy_collate([variant_to_dict(v) for v in records])
   369        14        328.0     23.4      0.0          batch['line_idx'] = np.array(line_ids)
   370        14   44412474.0 3172319.6     87.5          batch['preds'] = {k: df_to_np_dict(df) for k, df in six

`df_to_np_dict` is the bottleneck

## Buffer writing

Let's use buffer_size=5

In [12]:
tsv_writer =  SyncBatchWriter(TsvBatchWriter(output_dir / "preds.tsv"), buffer_size=5)
sp.score_variants(model = model_name, dl_args=dataloader_arguments, input_vcf=vcf_path, output_writers=tsv_writer, output_vcf=None)

Using downloaded and verified file: /home/avsec/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1


100%|██████████| 14/14 [00:06<00:00,  2.22it/s]


This took 6 seconds in total instead of 51. Is this still the bottleneck?

In [13]:
tsv_writer =  SyncBatchWriter(TsvBatchWriter(output_dir / "preds.tsv"), buffer_size=5)
%lprun -f sp.predict_snvs sp.score_variants(model = model_name, dl_args=dataloader_arguments, input_vcf=vcf_path, output_writers=tsv_writer, output_vcf=None)

Using downloaded and verified file: /home/avsec/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1


100%|██████████| 14/14 [00:09<00:00,  1.49it/s]


Timer unit: 1e-06 s

Total time: 13.4897 s
File: /home/avsec/workspace/kipoi/kipoi-veff/kipoi_veff/snv_predict.py
Function: predict_snvs at line 468

Line #      Hits         Time  Per Hit   % Time  Line Contents
   468                                           def predict_snvs(model,
   469                                                            dataloader,
   470                                                            vcf_fpath,
   471                                                            batch_size,
   472                                                            num_workers=0,
   473                                                            dataloader_args=None,
   474                                                            vcf_to_region=None,
   475                                                            vcf_id_generator_fn=default_vcf_id_gen,
   476                                                            evaluation_function=analyse_model_preds,
   477       

Yes. It still takes 63.1% of the time writing.

## Taking it from there

- use a separate thread to write to the file on line 656
  - question: is thread enough or do we need a separate process (using multiprocessing)?